# Algoritma *K-Nearest Neighbor*

## Mari berkenalan dengan Algoritma KNN

insert penjelasan here...

## Mengimpor Pustaka

Pada bagian ini akan diimpor beberapa pustaka yang akan digunakan untuk menyimulasikan Algoritma *K-Nearest Neighbor*.

In [1]:
import pandas as pd
import numpy as np

Selain itu, akan dipanggil juga modul KNN yang telah dibangun *from scratch*

In [2]:
from algorithm.knn import KNNAlgorithm

## Mengimpor *Dataset*

Pada bagian ini akan diimpor *dataset* yang sebelumnya telah terbagi menjadi `data_train.csv` dan `data_validation.csv`.

In [3]:
# Mengambil data train dan data validation
df_train = pd.read_csv("../data/data_train.csv")
df_validation = pd.read_csv("../data/data_validation.csv")

# *Pre-processing* Data

Tahap yang dilakukan meliputi pemisahan kolom target hingga melakukan standarisasi terhadap data sebelum dilakukan pemrosesan dengan Algoritma KNN.

In [4]:
# Melakukan pemisahan kolom target
# Pada bagian ini, dipilih fitur dengan nilai korelasi diatas 0.1 melalui hasil EDA
columns_to_exclude = ["blue", "wifi", "three_g", "int_memory", "sc_w", "clock_speed", "sc_h", "talk_time", "m_dep", "four_g", "n_cores", "fc", "pc", "dual_sim", "touch_screen", "mobile_wt", "price_range"]
x_train = df_train.drop(columns=columns_to_exclude)
y_train = df_train["price_range"]

x_test = df_validation.drop(columns=columns_to_exclude)
y_test = df_validation["price_range"]

In [5]:
# Dilakukan standarisasi dengan scaler yang dibuat mandiri
from utils.scaler import Scaler

scaler = Scaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Hasil Pemrosesan Algoritma KNN yang Dibangun

Berikut adalah hasil pemrosesan Algoritma KNN yang dibangun *from scratch*.

In [6]:
# Gunakan model KNN yang sebelumnya dibangun
knn_scratch = KNNAlgorithm(5)       # Gunakan nilai k = 5 sebagai nilai awal

# Lakukan fit model
knn_scratch.fit(x_train, y_train)

# Lakukan prediksi dengan data validation
y_pred_scratch = knn_scratch.predict(x_test)

In [7]:
# Pengujian kualitas model dengan metrik
from sklearn.metrics import accuracy_score, classification_report

print(classification_report(y_test, y_pred_scratch))
print("Akurasi : ", 100 * np.round(accuracy_score(y_test, y_pred_scratch), 5), "%")

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       142
           1       0.84      0.83      0.83       144
           2       0.83      0.83      0.83       155
           3       0.94      0.90      0.92       159

    accuracy                           0.88       600
   macro avg       0.88      0.88      0.88       600
weighted avg       0.88      0.88      0.88       600

Akurasi :  87.833 %


## Hasil Pemrosesan Algoritma KNN Pembanding

Hasil pemrosesan diatas akan dibandingkan dengan hasil yang diperoleh dari *library* scikit-learn

In [8]:
# Pemanggilan model KNN dari scikit-learn
from sklearn.neighbors import KNeighborsClassifier

knn_scikit = KNeighborsClassifier(n_neighbors=5)        # Dengan menggunakan nilai k yang sama
knn_scikit.fit(x_train, y_train)
y_pred_scikit = knn_scikit.predict(x_test)

In [9]:
# Pengujian kualitas model dengan metrik
print(classification_report(y_test, y_pred_scikit))
print("Akurasi : ", 100 * np.round(accuracy_score(y_test, y_pred_scikit), 5), "%")

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       142
           1       0.84      0.83      0.83       144
           2       0.83      0.83      0.83       155
           3       0.94      0.90      0.92       159

    accuracy                           0.88       600
   macro avg       0.88      0.88      0.88       600
weighted avg       0.88      0.88      0.88       600

Akurasi :  87.833 %


## Hasil yang diperoleh

Berdasarkan kedua hasil tersebut, diperoleh **hasil yang sama** antara nilai prediksi yang dihasilkan oleh model KNN yang dibangun *from scratch* dengan model yang dimiliki scikit-learn.

In [10]:
# Pengujian kualitas model dengan metrik
print(classification_report(y_pred_scratch, y_pred_scikit))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       151
           1       1.00      1.00      1.00       142
           2       1.00      1.00      1.00       155
           3       1.00      1.00      1.00       152

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



## Apakah sudah merupakan model yang terbaik?

Belum tentu! proses kalkulasi untuk mendapatkan model dengan akurasi yang tinggi memerlukan pemilihan nilai k yang baik. Melalui eksperimen diatas, telah dilakukan percobaan pemilihan nilai k = 5 dan diperoleh nilai akurasi sebesar 87.833 %. Angka ini tentu saja masih dapat ditingkatkan dengan melakukan pemilihan nilai k yang dapat memaksimalkan nilai akurasi.

## Memperkenalkan *Cross-validation*

Menemukan nilai terbaik untuk hyperparameter 'k' di *K-Nearest Neighbor* (KNN) melibatkan proses yang disebut *hyperparameter tuning*. Salah satu pendekatan yang umum adalah dengan menggunakan *cross-validation*.
Berikut adalah detail prosedurnya
1. Pemilihan Jumlah Subset (*Fold*)<br/> 
*Dataset* dibagi menjadi beberapa subset yang disebut "fold." Misalnya, dalam *5-fold cross-validation*, *dataset* dibagi menjadi 5 bagian. Proses pelatihan dan pengujian akan dilakukan sebanyak 5 kali, di mana setiap *fold* digunakan sebagai subset pengujian satu kali, dan sisanya digunakan sebagai subset pelatihan.<br/>
2. Pelatihan dan Pengujian Berulang<br/> 
Model pembelajaran mesin dilatih pada subset pelatihan dan diuji pada subset pengujian untuk setiap iterasi *cross-validation*. Proses ini dilakukan sebanyak jumlah *fold* yang telah ditentukan. <br/>
3. Perhitungan Metrik Kinerja<br/> 
Metrik kinerja seperti akurasi, presisi, *recall*, atau *F1-score* dihitung untuk setiap iterasi *cross-validation*. Metrik ini memberikan gambaran tentang seberapa baik model berkinerja pada berbagai subset data.

Berikut adalah prosedur yang dilakukan untuk melakukan *cross-validation* dengan menggunakan *library* milik scikit-learn. Secara umum proses yang dilakukan adalah *Grid Search* dengan rentang nilai 1 hingga 500.

In [11]:
# Impor library GridSearchCV milik sklearn
from sklearn.model_selection import GridSearchCV

# Pengaturan parameter pencarian menggunakan gridSearch
param_grid = {'n_neighbors': list(range(1, 500)), 'metric': ['euclidean']}

# Membuat KNN classifier baru
knn = KNeighborsClassifier()

# Melakukan gridSearch dengan cross-validation untuk 5 fold
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Mendapatkan parameter dan model KNN terbaik
best_params = grid_search.best_params_
best_knn = grid_search.best_estimator_

Setelah mendapatkan nilai k dan model terbaik, mari kembali lakukan pengujian menggunakan Algoritma KNN yang telah dibangun sebelumnya

In [12]:
# Gunakan model KNN terbaik
y_pred = best_knn.predict(x_test)

In [21]:
# Pengujian kualitas model dengan metrik
print(classification_report(y_test, y_pred))
print("Akurasi : ", 100 * np.round(accuracy_score(y_test, y_pred), 5), "%")
print("Nilai hyperparameter terbaik:", best_params)

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       142
           1       0.85      0.87      0.86       144
           2       0.82      0.85      0.84       155
           3       0.96      0.86      0.91       159

    accuracy                           0.89       600
   macro avg       0.89      0.89      0.89       600
weighted avg       0.89      0.89      0.89       600

Akurasi :  88.5 %
Nilai hyperparameter terbaik: {'metric': 'euclidean', 'n_neighbors': 40}


Terlihat bahwa dengan melakukan cross-validation pada rentang nilai k dari 1 hingga 500, diperoleh nilai k terbaik adalah 40 dengan nilai akurasi 88.5 %. Angka ini meningkat dari akurasi sebelumnya!

## Penyimpanan dan *Load* Model

Agar model dapat digunakan kembali, maka model harus dapat disimpan dan di-*load*. Berikut adalah implementasi penyimpanan model yang dilakukan menggunakan *library* pickle.

In [14]:
import pickle

# Menyimpan model dalam pkl
model_pkl_file = "models/knn_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(best_knn, file)

Untuk membuktikan bahwa model berhasil tersimpan, berikut adalah pembuktian pemanggilan kembali hasil prediksi model yang disimpan

In [15]:
# Load kembali model dari pkl
with open(model_pkl_file, 'rb') as file:  
    model = pickle.load(file)

# Melakukan prediksi dengan model tersebut
y_pickle = model.predict(x_test)

# Menguji hasil akurasi model
print(classification_report(y_test, y_pickle)) 

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       142
           1       0.85      0.87      0.86       144
           2       0.82      0.85      0.84       155
           3       0.96      0.86      0.91       159

    accuracy                           0.89       600
   macro avg       0.89      0.89      0.89       600
weighted avg       0.89      0.89      0.89       600



Terbukti bahwa model pkl yang disimpan berhasil untuk digunakan kembali.

## [Bonus] Submisi Kaggle

Bagian ini dikhususkan untuk pemrosesan data dan penggunaan model yang dibuat sebagai dasar membuat submisi pada Kaggle.

In [27]:
# Impor dataset
test_data = pd.read_csv('../data/test.csv')

columns_to_exclude_new = ["id", "blue", "wifi", "three_g", "int_memory", "sc_w", "clock_speed", "sc_h", "talk_time", "m_dep", "four_g", "n_cores", "fc", "pc", "dual_sim", "touch_screen", "mobile_wt"]
features = test_data.drop(columns=columns_to_exclude_new)

# Gunakan model KNN yang sebelumnya dibangun
knn_kaggle = KNeighborsClassifier(n_neighbors=40)

# Lakukan fit model
knn_kaggle.fit(x_train, y_train)

# Lakukan prediksi dengan data validation
scaler_final = Scaler()
feature_test = scaler.fit_transform(features)
predictions = knn_kaggle.predict(feature_test)

# Membuat dataframe hasil
result_df = pd.DataFrame({'id': test_data['id'], 'price_range': predictions})

# Menyimpan dataframe dalam csv
result_df.to_csv('../result/predictions-knn9.csv', index=False)